In [80]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error
import joblib

In [81]:
#doc file du lieu
data = pd.read_csv("student.csv")
data

,StudentID,Name,Gender,AttendanceRate,StudyHoursPerWeek,PreviousGrade,ExtracurricularActivities,ParentalSupport,FinalGrade
0,1,John,Male,85,15,78,1,High,80
1,2,Sarah,Female,90,20,85,2,Medium,87
2,3,Alex,Male,78,10,65,0,Low,68
3,4,Michael,Male,92,25,90,3,High,92
4,5,Emma,Female,88,18,82,2,Medium,85
5,6,Olivia,Female,95,30,88,1,High,90
6,7,Daniel,Male,70,8,60,0,Low,62
7,8,Sophia,Female,85,17,77,1,Medium,78
8,9,James,Male,82,12,70,2,Low,72
9,10,Isabella,Female,91,22,86,3,High,88


In [82]:
data.columns

Index(['StudentID', 'Name', 'Gender', 'AttendanceRate', 'StudyHoursPerWeek',
       'PreviousGrade', 'ExtracurricularActivities', 'ParentalSupport',
       'FinalGrade'],
      dtype='object')

In [83]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   StudentID                  10 non-null     int64 
 1   Name                       10 non-null     object
 2   Gender                     10 non-null     object
 3   AttendanceRate             10 non-null     int64 
 4   StudyHoursPerWeek          10 non-null     int64 
 5   PreviousGrade              10 non-null     int64 
 6   ExtracurricularActivities  10 non-null     int64 
 7   ParentalSupport            10 non-null     object
 8   FinalGrade                 10 non-null     int64 
dtypes: int64(6), object(3)
memory usage: 852.0+ bytes


In [84]:
#loai bo ID va ten boi vi no khong lam anh huong den kq 
data.drop(columns=['StudentID', 'Name','Gender'], inplace=True)

In [85]:
# ma hoa phân loại từ sự hỗ trợ của bố mẹ low=0,medium=1,high=2
from sklearn.preprocessing import OrdinalEncoder

list = [['Low', 'Medium', 'High']]

PL = OrdinalEncoder(categories=list)
data['ParentalSupport'] = PL.fit_transform(data[['ParentalSupport']])

In [86]:
data.head()

,AttendanceRate,StudyHoursPerWeek,PreviousGrade,ExtracurricularActivities,ParentalSupport,FinalGrade
0,85,15,78,1,2.0,80
1,90,20,85,2,1.0,87
2,78,10,65,0,0.0,68
3,92,25,90,3,2.0,92
4,88,18,82,2,1.0,85


In [87]:
#lay du lieu input va output
X =data.iloc[:,:-1]
Y = data.iloc[:,-1]


In [88]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train ,Y_test=train_test_split(X,Y,train_size=0.8,test_size = 0.2,random_state=42)

In [89]:
#hoiquytuyentinh
regressor = LinearRegression()
regressor.fit(X_train, Y_train)
y_pred_1=regressor.predict(X_test)
joblib.dump(regressor, 'linear_model.pkl');




In [90]:
#so sanh data train va thuc te
pd.DataFrame({"y": Y_test,"y_pred_1": y_pred_1})

,y,y_pred_1
8,72,73.303705
1,87,87.489091


In [91]:
#hoiquylasso
lassoreg = Lasso(alpha = 0.01)
lassoreg.fit(X_train,Y_train)
y_pred_2=lassoreg.predict(X_test)
joblib.dump(lassoreg, 'lasso_model.pkl')

['lasso_model.pkl']

In [92]:
pd.DataFrame({"y": Y_test,"y_pred_2": y_pred_2})

,y,y_pred_2
8,72,73.221908
1,87,87.421636


In [93]:
#neural
mlp_model_neural = MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=500, random_state=0)

mlp_model_neural.fit(X_train, Y_train)
y_pred_3=mlp_model_neural.predict(X_test)


In [94]:
pd.DataFrame({"y": Y_test,"y_pred_3": y_pred_3})

,y,y_pred_3
8,72,79.973100
1,87,94.058446


In [95]:
def evaluate_model(y_test, y_pred, model_name):
    mse = mean_squared_error(y_test, y_pred)# lỗi trung bình
    mae=mean_absolute_error(y_test,y_pred)# lỗi tuyệt đối trung bình
    r2 = r2_score(y_test, y_pred)# hệ số xác định
    rmse=np.sqrt(mse)#lỗi bình phương trung bình càng thấp càng tốt
    print(f"{model_name} MSE: {mse:.3f}, MAE: {mae:.3f}, R2 Score: {r2:.3f}, RMSE: {rmse:.3f}")

In [96]:
evaluate_model(Y_test,y_pred_1, "Linear Regression")
evaluate_model(Y_test,y_pred_2, "Lasso Regression")
evaluate_model(Y_test,y_pred_3, "Neural Network")


Linear Regression MSE: 0.969, MAE: 0.896, R2 Score: 0.983, RMSE: 0.985
Lasso Regression MSE: 0.835, MAE: 0.822, R2 Score: 0.985, RMSE: 0.914
Neural Network MSE: 56.696, MAE: 7.516, R2 Score: -0.008, RMSE: 7.530


In [97]:
# Tạo mô hình Bagging1
model_1 = regressor
bagging_model1 = BaggingRegressor(model_1,n_estimators=10,random_state=0)
bagging_model1.fit(X_train,Y_train)
bagging_1=bagging_model1.predict(X_test)



In [98]:
pd.DataFrame({"y": Y_test,"y_bagging_1": bagging_1})

,y,y_bagging_1
8,72,73.982243
1,87,85.805362


In [99]:
# Tạo mô hình Bagging2
model_2=lassoreg
bagging_model2 = BaggingRegressor(model_2,n_estimators=10,random_state=0)
bagging_model2.fit(X_train,Y_train)
bagging_2=bagging_model2.predict(X_test)


c:\Users\MY COMPUTER\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.766e-01, tolerance: 4.880e-02
  model = cd_fast.enet_coordinate_descent(
c:\Users\MY COMPUTER\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.039e-01, tolerance: 5.775e-02
  model = cd_fast.enet_coordinate_descent(
c:\Users\MY COMPUTER\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterati

In [100]:
pd.DataFrame({"y": Y_test,"y_bagging_2": bagging_2})

,y,y_bagging_2
8,72,74.241201
1,87,86.080199


In [101]:
# Tạo mô hình Bagging3
model_3=mlp_model_neural
bagging_model3 = BaggingRegressor(model_3,n_estimators=10,random_state=0)
bagging_model3.fit(X_train,Y_train)
bagging_3=bagging_model3.predict(X_test)

In [102]:
pd.DataFrame({"y": Y_test,"y_bagging_3": bagging_3})

,y,y_bagging_3
8,72,81.942345
1,87,95.562469


In [103]:
evaluate_model(Y_test,bagging_1, "Linear")
evaluate_model(Y_test,bagging_2, "Lasso ")
evaluate_model(Y_test,bagging_3, "Neural")


Linear MSE: 2.678, MAE: 1.588, R2 Score: 0.952, RMSE: 1.637
Lasso  MSE: 2.935, MAE: 1.581, R2 Score: 0.948, RMSE: 1.713
Neural MSE: 86.083, MAE: 9.252, R2 Score: -0.530, RMSE: 9.278


In [115]:
models=[model_1,model_2,model_3]
#input 
new_input=([[100,100,100,3,100]])

predictions = [model.predict(new_input) for model in models]
final_prediction = sum(predictions) / len(models)
out= final_prediction if final_prediction > 0 else 0
print(f"Dự đoán cuối cùng: {out}")





Dự đoán cuối cùng: [93.52824899]


c:\Users\MY COMPUTER\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
c:\Users\MY COMPUTER\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but Lasso was fitted with feature names
  warnings.warn(
c:\Users\MY COMPUTER\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MLPRegressor was fitted with feature names
  warnings.warn(
